# Packages Installs, imports, and presets

In [52]:
#from scapy.all import *
import pandas as pd
#import seaborn as sns
import numpy as np
from collections import defaultdict
import os

In [53]:
INPUT_SIZE = (256,256,3)

In [54]:
df = pd.read_parquet('data/iec104.parquet')

In [57]:
pkts = df.groupby('flow').count()['tv_usec']

In [58]:
scipy.stats.mstats.mquantiles(pkts,prob=[0.25,0.5,0.75,0.9,0.99])

array([ 3.   ,  7.   , 10.   , 11.   , 18.018])

## Removing missing features

In [44]:
# df=df.drop([c for c,v in ((df==-1).sum() == len(df)).items() if v==True],axis=1)

In [45]:
df.shape

(518377, 1667)

In [46]:
df['tv_sec']=df['tv_sec'].astype(int)
df['tv_usec']=df['tv_usec'].astype(int)

# Model training

## Dataset preparation

In [47]:
df[['label']].value_counts()

label        
normal           400317
c_rd_na_1_DoS     20167
c_rp_na_1_DoS     19949
c_sc_na_1_DoS     16615
c_rp_na_1         11517
c_se_na_1_DoS     10854
c_rd_na_1         10821
c_ci_na_1_DoS      9146
c_se_na_1          6960
c_sc_na_1          6767
c_ci_na_1          5264
Name: count, dtype: int64

In [48]:
df[['label']]

,label
flow,
1_192.168.1.13_192.168.1.29_2404_46773_TCP_c_rd_na_1_c_rd_na_1,c_rd_na_1
1_192.168.1.13_192.168.1.29_2404_46773_TCP_c_rd_na_1_c_rd_na_1,c_rd_na_1
1_192.168.1.13_192.168.1.29_2404_46773_TCP_c_rd_na_1_c_rd_na_1,c_rd_na_1
1_192.168.1.13_192.168.1.29_2404_46773_TCP_c_rd_na_1_c_rd_na_1,c_rd_na_1
1_192.168.1.13_192.168.1.29_2404_46773_TCP_c_rd_na_1_c_rd_na_1,c_rd_na_1
...,...
1_192.168.1.29_192.168.1.22_46871_2404_TCP_c_sc_na_1_c_sc_na_1,c_sc_na_1
1_192.168.1.29_192.168.1.22_46871_2404_TCP_c_sc_na_1_c_sc_na_1,c_sc_na_1
1_192.168.1.29_192.168.1.22_46871_2404_TCP_c_sc_na_1_c_sc_na_1,c_sc_na_1


In [49]:
from feature_engine.encoding import OrdinalEncoder

In [50]:
od = OrdinalEncoder(encoding_method='arbitrary')
od.fit(df[['label']])
df[['label']]= od.transform(df[['label']])

In [51]:
TRAIN_RATE= 0.8
import random
indexes = list(set(df.index))
random.Random(1).shuffle(indexes)
TRAIN_SIZE = int(len(indexes)*TRAIN_RATE)
TEST_SIZE = len(indexes)-TRAIN_SIZE
TRAIN_SAMPLES = indexes[:TRAIN_SIZE]
TEST_SAMPLES = indexes[:TEST_SIZE]
df_train_initial = df.loc[TRAIN_SAMPLES]
df_test = df.loc[TEST_SAMPLES]
# df_test = df.iloc[TRAIN_SAMPLES:]
# del df

In [52]:
indexes = list(set(df_train_initial.index))
random.Random(1).shuffle(indexes)
TRAIN_SIZE = int(len(indexes)*TRAIN_RATE)
TEST_SIZE = len(indexes)-TRAIN_SIZE
TRAIN_SAMPLES = indexes[:TRAIN_SIZE]
TEST_SAMPLES = indexes[:TEST_SIZE]
df_train = df_train_initial.loc[TRAIN_SAMPLES]
df_validation = df_train_initial.loc[TEST_SAMPLES]

In [53]:
indexes[:3]

['1_192.168.1.28_192.168.1.24_43067_2404_TCP_c_rp_na_1_DoS_c_rp_na_1_DoS',
 '110_192.168.1.20_192.168.1.21_49389_2404_TCP_c_ci_na_1_DoS_normal',
 '196_192.168.1.20_192.168.1.13_49672_2404_TCP_c_sc_na_1_DoS_normal']

In [54]:
df_train.shape, df_validation.shape, df_test.shape

((319038, 1667), (95907, 1667), (83735, 1667))

In [55]:
len(set(df_train.index))

30756

In [56]:
display(df_train['label'].value_counts())
display(df_validation['label'].value_counts())
display(df_test['label'].value_counts())

label
1     243295
3      12729
7      12658
6      10702
4       7241
5       7153
0       6902
10      5876
8       4632
9       4466
2       3384
Name: count, dtype: int64

label
1     77307
7      3096
3      3013
6      2522
5      1855
0      1769
4      1748
10     1361
8      1194
9      1112
2       930
Name: count, dtype: int64

label
1     60019
3      4252
7      3883
6      3327
0      2292
4      2224
5      2090
10     1882
9      1377
8      1360
2      1029
Name: count, dtype: int64

In [57]:
df_train.to_parquet('data/iec104_train.parquet')
df_validation.to_parquet('data/iec104_validation.parquet')
df_test.to_parquet('data/iec104_test.parquet')